In [1]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf

In [2]:
dataset = pd.read_csv(r'../../data/processed/charity_main_cleaned.csv')
ladataset = pd.read_excel(r'../../data/raw/Local_government_finance_data_download.xlsx',sheet_name='Spending power totals')
housing_net_supply_df = pd.read_excel('../../data/raw/Local-authority-housing-supply.xlsx', sheet_name='Net supply')
housing_stock_df = pd.read_excel('../../data/raw/Local-authority-housing-supply.xlsx', sheet_name='Housing stock')
population = pd.read_excel('../../data/raw/population_by_age.xlsx', sheet_name='Age bands')

C:\Users\User\AppData\Local\Temp\ipykernel_31708\925063635.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  dataset = pd.read_csv(r'../../data/processed/charity_main_cleaned.csv')


In [3]:
# Define function to get financial year
def get_financial_year(date):
    if pd.isna(date):
        return np.nan
    return date.year if date.month >= 4 else date.year - 1

# --- ensure dates are in datetime ---
dataset['date_of_registration'] = pd.to_datetime(dataset['date_of_registration'], errors='coerce')
dataset['date_of_removal'] = pd.to_datetime(dataset['date_of_removal'], errors='coerce')

dataset['registration_year'] = dataset['date_of_registration'].dt.to_period('Y')
dataset['removal_year'] = dataset['date_of_removal'].dt.to_period('Y')

dataset['registration_month'] = dataset['date_of_registration'].dt.to_period('M')
dataset['removal_month'] = dataset['date_of_removal'].dt.to_period('M')

dataset['registration_fy'] = dataset['date_of_registration'].apply(get_financial_year)
dataset['removal_fy'] = dataset['date_of_removal'].apply(get_financial_year)

In [4]:
# Step 1: Drop rows with missing classification
classification_df = dataset[['registered_charity_number', 'classification_description']].dropna()

# Step 2: Create binary indicator (1) for each classification
classification_df['value'] = 1

# Step 3: Pivot to wide format with binary columns
classification_dummies = classification_df.pivot_table(
    index='registered_charity_number',
    columns='classification_description',
    values='value',
    aggfunc='max',
    fill_value=0
)

# Step 4: Optional – rename columns for consistency (e.g., no spaces or special chars)
classification_dummies.columns = [
    f"classification_{str(col).replace(' ', '_').replace('-', '_').lower()}"
    for col in classification_dummies.columns
]

# Step 5: Reset index and merge with original dataset
classification_dummies = classification_dummies.reset_index()
dataset = dataset.drop_duplicates(subset='registered_charity_number')  # ensure one row per charity
dataset = dataset.merge(classification_dummies, on='registered_charity_number', how='left')

# Identify classification dummy columns (typically start with 'classification_')
category_cols = [col for col in dataset.columns if col.startswith('classification_')]

# Fill NaNs with 0 in those columns only
dataset[category_cols] = dataset[category_cols].fillna(0)


In [5]:
# Step 1: Calculate total 2024 housing stock per council (Local authority only)
stock_2024 = (
    housing_stock_df
    .groupby(['Authority_code', 'Authority_name'], as_index=False)['LA_number']
    .sum()
    .rename(columns={'LA_number': 'Stock_2024'})
)

# Step 2: Prepare Net Supply Data
net = housing_net_supply_df.copy()
net = net[net['Year'].astype(str).str.match(r'^\d{4}/\d{2}$')]  # Keep only year-formatted rows
net['Year'] = net['Year'].str[:4].astype(int)  # Convert '2015/16' → 2015

# Pivot to Local Authority x Year format
net_pivot = net.pivot(index='LA_code', columns='Year', values='Net_additions').fillna(0)
net_pivot = net_pivot[sorted(net_pivot.columns, reverse=True)]  # descending order

# Step 3: Merge with 2024 stock
stock = stock_2024.set_index('Authority_code')
net_pivot = net_pivot.reindex(stock.index)  # align index

# Step 4: Calculate total stock backwards from 2024
stock_by_year = pd.DataFrame(index=net_pivot.index)
stock_by_year[2024] = stock['Stock_2024']

for year in sorted(net_pivot.columns, reverse=True):
    if year < 2024:
        stock_by_year[year] = stock_by_year[year + 1] - net_pivot[year]

# Step 5: Merge back Authority name and reshape
stock_by_year = stock_by_year.merge(stock[['Authority_name']], left_index=True, right_index=True)
stock_by_year_reset = stock_by_year.reset_index().melt(id_vars=['Authority_code', 'Authority_name'], 
                                                       var_name='Year', value_name='Housing stock')

# Final pivot
stock_by_year_pivot = stock_by_year_reset.pivot(index=['Authority_code', 'Authority_name'], 
                                                 columns='Year', values='Housing stock').sort_index(axis=1)

# Reset index, rename, and set it again
stock_by_year_pivot = stock_by_year_pivot.reset_index().rename(columns={'Authority_code': 'ONS code'})
stock_by_year_pivot = stock_by_year_pivot.set_index(['ONS code', 'Authority_name'])


In [7]:
# Filter: Remove NaN and keep only rows where 'Year' matches 'YYYY/YY'
ladataset_filtered = ladataset[
    ladataset['Year'].notna() & ladataset['Year'].str.match(r'^\d{4}/\d{2}$')
]
ladataset_filtered['Year'] = ladataset_filtered['Year'].str[:4].astype(int)

# Pivot
ladataset_pivot = ladataset_filtered.pivot_table(
    index=['ONS code', 'Local authority', 'Measure'],
    columns='Year',
    values='£ millions, cash terms',
    aggfunc='sum'
).reset_index()

# Group by 'con_code' only
population_summary = (
    population
    .groupby('con_code', as_index=False)
    .agg({'con_name': 'first', 'con_number': 'sum'})
    .rename(columns={'con_name': 'Local authority', 'con_number': 'Total Population'})
)

# Display the result
print(population_summary.head())


    con_code           Local authority  Total Population
0  E14001063                 Aldershot            119256
1  E14001064       Aldridge-Brownhills             93720
2  E14001065  Altrincham and Sale West            101272
3  E14001066              Amber Valley             90793
4  E14001067   Arundel and South Downs             98053


C:\Users\User\AppData\Local\Temp\ipykernel_31708\3580503092.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ladataset_filtered['Year'] = ladataset_filtered['Year'].str[:4].astype(int)


In [8]:
# Filter for Core Spending Power (CSP)
csp = ladataset_pivot.copy()

# Keep only relevant years
csp = csp.set_index(['ONS code', 'Local authority']).drop(columns='Measure')

# Calculate % change from 2015
csp['financial_distress'] = (csp[2024] - csp[2015]) / csp[2015]

fd = csp[['financial_distress']].reset_index()

fd.columns.name = None
housing_stock_processed = housing_stock_df[housing_stock_df['Tenure'] == 'Local authority'][['Authority_code', 'LA_number']]
fd_new = fd.merge(housing_stock_processed, left_on='ONS code', right_on='Authority_code', how='inner')
fd_new = fd_new.drop(columns='Authority_code')
fd_new = fd_new.rename(columns={'LA_number': 'LA Housing stock 2024'})


In [9]:
# Step 1: Normalise both variables
fd_new['financial_distress_norm'] = (
    (fd_new['financial_distress'] - fd_new['financial_distress'].min()) /
    (fd_new['financial_distress'].max() - fd_new['financial_distress'].min())
)

fd_new['LA Housing stock 2024_norm'] = (
    (fd_new['LA Housing stock 2024'] - fd_new['LA Housing stock 2024'].min()) /
    (fd_new['LA Housing stock 2024'].max() - fd_new['LA Housing stock 2024'].min())
)

# Step 2: Avoid division by zero and calculate treatment
fd_new['treatment'] = np.where(
    fd_new['LA Housing stock 2024_norm'] != 0,
    fd_new['financial_distress_norm'] / fd_new['LA Housing stock 2024_norm'],
    0
)
fd_new.rename(columns={'Local authority': 'local_authority'}, inplace=True)

In [10]:
# Step 2: Filter valid rows
removed = dataset[dataset['removal_fy'].notnull() & dataset['local_authority'].notnull()].copy()
removed['removal_fy'] = removed['removal_fy'].astype(int)

# Step 3: Group by financial year and local authority
removed_by_fy_la = (
    removed
    .groupby(['local_authority', 'removal_fy'])
    .size()
    .unstack(fill_value=0)
    .sort_index(axis=1)
)

# Step 4: Keep only FY 2015–2024
fy_years = list(range(2015, 2025))
removed_by_fy_la = removed_by_fy_la[removed_by_fy_la.columns.intersection(fy_years)]

# Step 5: Total and sort
removed_by_fy_la['Total'] = removed_by_fy_la.sum(axis=1)
removed_by_fy_la = removed_by_fy_la.sort_values(by='Total', ascending=False)

# Step 6: Preview top 10
print("Removed Charities per Financial Year per Local Authority (FY 2015–2024):")
print(removed_by_fy_la.head(10))

Removed Charities per Financial Year per Local Authority (FY 2015–2024):
removal_fy       2015  2016  2017  2018  2019  2020  2021  2022  2023  2024  \
local_authority                                                               
North Yorkshire    66    88    79    94    89    54    87    81    51    81   
Westminster        63    80    64    92   111    54    63    70    68    64   
Somerset           70    41    74    59    71    57    65    46    52    59   
Birmingham         52    56    57    72    89    38    67    60    43    55   
Cornwall           47    57    58    63    62    48    69    57    37    55   
Buckinghamshire    41    58    57    63    69    42    60    46    47    51   
Wiltshire          40    57    50    62    68    54    49    54    47    48   
Camden             38    53    49    51    76    35    61    42    62    56   
Barnet             47    39    53    44    76    32    37    32    42    42   
Dorset             49    50    41    45    43    25    58 

In [11]:
# Step 1: Define year range
years = list(range(2015, 2025))

# Step 2: Calculate percentage change across years
removed_pct_change = removed_by_fy_la[years].pct_change(axis=1) * 100

# Step 3: Round for readability
removed_pct_change = removed_pct_change.round(2)
removed_pct_change

removal_fy,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
local_authority,,,,,,,,,,
North Yorkshire,NaN,33.33,-10.23,18.99,-5.32,-39.33,61.11,-6.90,-37.04,58.82
Westminster,NaN,26.98,-20.00,43.75,20.65,-51.35,16.67,11.11,-2.86,-5.88
Somerset,NaN,-41.43,80.49,-20.27,20.34,-19.72,14.04,-29.23,13.04,13.46
Birmingham,NaN,7.69,1.79,26.32,23.61,-57.30,76.32,-10.45,-28.33,27.91
Cornwall,NaN,21.28,1.75,8.62,-1.59,-22.58,43.75,-17.39,-35.09,48.65
...,...,...,...,...,...,...,...,...,...,...
Orkney Islands,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,inf
South Lanarkshire,NaN,-100.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"Armagh City, Banbridge and Craigavon",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,inf


In [12]:
removed_long = (
    removed_pct_change
    .reset_index()
    .melt(id_vars='local_authority', var_name='Year', value_name='removed_rate')
)
removed_long['Year'] = removed_long['Year'].astype(int)

# Add treatment, post, and controls
panel_df = removed_long.merge(fd_new, on='local_authority', how='left')
panel_df['post'] = (panel_df['Year'] >= 2021).astype(int)
panel_df['treatment_post'] = panel_df['treatment'] * panel_df['post']

panel_df = panel_df[np.isfinite(panel_df['removed_rate'])]
panel_df['removed_rate'].dropna(inplace=True)
panel_df.drop(columns=['ONS code'], inplace=True)

In [14]:
panel_df

,local_authority,Year,removed_rate,financial_distress,LA Housing stock 2024,financial_distress_norm,LA Housing stock 2024_norm,treatment,post,treatment_post
350,North Yorkshire,2016,33.33,NaN,NaN,NaN,NaN,NaN,0,NaN
351,Westminster,2016,26.98,0.355025,12207.0,0.674481,0.207556,3.249631,0,0.000000
352,Somerset,2016,-41.43,NaN,NaN,NaN,NaN,NaN,0,NaN
353,Birmingham,2016,7.69,0.466262,58813.0,0.858823,1.000000,0.858823,0,0.000000
354,Cornwall,2016,21.28,0.490814,10381.0,0.899511,0.176509,5.096133,0,0.000000
...,...,...,...,...,...,...,...,...,...,...
3473,Fife,2024,-100.00,NaN,NaN,NaN,NaN,NaN,1,NaN
3474,Highland,2024,0.00,NaN,NaN,NaN,NaN,NaN,1,NaN
3476,Dumfries and Galloway,2024,-100.00,NaN,NaN,NaN,NaN,NaN,1,NaN
3481,Isles of Scilly,2024,-100.00,0.438637,115.0,0.813043,0.001955,415.804279,1,415.804279


In [13]:
model = smf.ols('removed_rate ~ post + treatment + C(local_authority)+ treatment:post', data=panel_df).fit()
summary = model.summary()
print(summary.tables[0])  # Overview: R², F-stat, etc.
print(summary.tables[1].as_text())  # Full coefficient table


                            OLS Regression Results                            
Dep. Variable:           removed_rate   R-squared:                       0.072
Model:                            OLS   Adj. R-squared:                 -0.045
Method:                 Least Squares   F-statistic:                    0.6172
Date:                Wed, 02 Jul 2025   Prob (F-statistic):               1.00
Time:                        11:08:28   Log-Likelihood:                -14842.
No. Observations:                2516   AIC:                         3.025e+04
Df Residuals:                    2233   BIC:                         3.190e+04
Df Model:                         282                                         
Covariance Type:            nonrobust                                         
                                                                coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------